In [44]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd
from multiprocessing import Pool
from sys import getsizeof
from keras.utils import to_categorical
from sklearn.preprocessing import normalize, Normalizer

# Prepare data for training

## Read the csv file for training

In [158]:
######### Use pandas to read "bridge.csv","combats.csv" file #########


bridge_dfN = pd.read_csv('N.csv')
bridge_dfS = pd.read_csv('S.csv')  
bridge_dfE = pd.read_csv('E.csv')  
bridge_dfW = pd.read_csv('W.csv')

#bridge_dfN = pd.DataFrame(df_N)
#bridge_dfS = pd.DataFrame(df_S)
#bridge_dfE = pd.DataFrame(df_E)
#bridge_dfW = pd.DataFrame(df_W)

bridge_dfN = bridge_dfN[:30000]
bridge_dfS = bridge_dfS[:30000]
bridge_dfE = bridge_dfE[:30000]
bridge_dfW = bridge_dfW[:30000]

bridge_dfN = bridge_dfN.drop('lin_file',axis = 1)
bridge_dfN = bridge_dfN.drop('board_num',axis = 1)
bridge_dfS = bridge_dfS.drop('lin_file',axis = 1)
bridge_dfS = bridge_dfS.drop('board_num',axis = 1)
bridge_dfE = bridge_dfE.drop('lin_file',axis = 1)
bridge_dfE = bridge_dfE.drop('board_num',axis = 1)
bridge_dfW = bridge_dfW.drop('lin_file',axis = 1)
bridge_dfW = bridge_dfW.drop('board_num',axis = 1)


bridge_dfN['direction'] = 'N'
bridge_dfS['direction'] = 'S'
bridge_dfE['direction'] = 'E'
bridge_dfW['direction'] = 'W'
'''
bridge_dfN_test = bridge_dfN[500000:505000]
bridge_dfS_test = bridge_dfS[500000:505000]
bridge_dfE_test = bridge_dfE[500000:505000]
bridge_dfW_test = bridge_dfW[500000:505000]
'''




bridge_df = pd.concat([bridge_dfN, bridge_dfS], ignore_index=True)
bridge_df = pd.concat([bridge_df, bridge_dfE], ignore_index=True)
bridge_df = pd.concat([bridge_df, bridge_dfW], ignore_index=True)
#bridge_df = bridge_df.astype('int')
#df = pd.read_csv('combats.csv') 
#combats_df = df
#print(combats_df.iloc[:,:])
#print(len(combats_df.iloc[:,:]))
print(len(bridge_dfN.iloc[:,:]))
print(len(bridge_dfS.iloc[:,:]))
print(len(bridge_dfE.iloc[:,:]))
print(len(bridge_dfW.iloc[:,:]))

print(len(bridge_df.iloc[:,:]))

30000
30000
30000
30000
120000


In [23]:
test = []
aaa = [1, 2, 3]
bbb = [4, 5, 6]
test.append(aaa)
test.append(bbb)
test

[[1, 2, 3], [4, 5, 6]]

In [175]:
#print(bridge_dfW)  #set seq = 30num

train_data = []
train_data_without_feature = []

for i in range(len(bridge_df.iloc[:,:])):
    if bridge_df.iloc[i,1] == 'none':
        bridge_df.iloc[i,1] = 0
    elif bridge_df.iloc[i,1] == 'NS':
        bridge_df.iloc[i,1] = 1
    elif bridge_df.iloc[i,1] == 'EW':
        bridge_df.iloc[i,1] = 2
    elif bridge_df.iloc[i,1] == 'both':
        bridge_df.iloc[i,1] = 3
    
    temp_list = []
    without_feature = []
    temp_list.append(bridge_df.iloc[i,0])
    without_feature.append(bridge_df.iloc[i,0])
    temp_list.append(bridge_df.iloc[i,1])
    without_feature.append(bridge_df.iloc[i,1])
    temp_list.append(bridge_df.iloc[i,2])
    without_feature.append(bridge_df.iloc[i,2])
    temp_list.append(bridge_df.iloc[i,3])
    without_feature.append(bridge_df.iloc[i,3])
    temp_list.append(bridge_df.iloc[i,5])
    without_feature.append(bridge_df.iloc[i,5])

    split_list6 = bridge_df.iloc[i,6].split('|')
    for j in range(4):
        temp_list.append(split_list6[j])
    '''
    split_list7 = bridge_df.iloc[i,7].split('|')
    for j in range(4):
        temp_list.append(split_list7[j])
    '''
    split_list9 = bridge_df.iloc[i,9].split('|')    
    for j in range(4):
        suit = [0]*13
        for k in range(len(split_list9[j])):
            if split_list9[j][k] == 'A':
                suit[1] = 1
            elif split_list9[j][k] == 'K':
                suit[0] = 1
            elif split_list9[j][k] == 'Q':
                suit[12] = 1
            elif split_list9[j][k] == 'J':
                suit[11] = 1
            elif split_list9[j][k] == 'T':
                suit[10] = 1
            elif split_list9[j][k].isdigit():
                suit[int(split_list9[j][k])] = 1
            else:
                suit[k] = -1
        without_feature = without_feature + suit
        temp_list = temp_list + suit

    split_list10 = bridge_df.iloc[i,10].split('|')
    for j in range(4):
        temp_list.append(split_list10[j])
    temp_list.append(bridge_df.iloc[i,11])

    split_list12 = bridge_df.iloc[i,12].split('|')
    for j in range(4):
        temp_list.append(split_list12[j])
    split_list13 = bridge_df.iloc[i,13].split('|')
    for j in range(4):
        temp_list.append(split_list13[j])
    split_list14 = bridge_df.iloc[i,14].split('|')
    for j in range(4):
        temp_list.append(split_list14[j])
    split_list15 = bridge_df.iloc[i,15].split('|')
    for j in range(4):
        temp_list.append(split_list15[j])

    temp_list.append(bridge_df.iloc[i,16])
    temp_list.append(bridge_df.iloc[i,17])
    if(bridge_df.iloc[i,18] == 'N'):
        temp_list.append(0)
    elif(bridge_df.iloc[i,18] == 'S'):
        temp_list.append(1)
    elif(bridge_df.iloc[i,18] == 'E'):
        temp_list.append(2)
    elif(bridge_df.iloc[i,18] == 'W'):
        temp_list.append(3)
    else:
        temp_list.append(-1)
        
    order = bridge_df.iloc[i,5]
    #print('order:', order)
    split_list = bridge_df.iloc[i,4].split('|')
    #print('len: ', len(split_list))
    if(len(split_list)%4-order >= 0):
        remain = 1
    else:
        remain = 0
    add_bidding = int(len(split_list)/4) + remain #can cut into how many examples for one bidding seqence
    #example = []
    
    for ex in range(add_bidding):
        temp = 0
        temp_list_bidding =[-1]*12
        end_1 = ex*4 + order - 1 # split_list[end_1] is the bidding choice of this player in this round 
        #print('end_1: ', end_1)
        #lenth_1 = len(split_list)-1
        for j in range(end_1):
            if(j>11):
                break
            split_list[end_1-j-1] = split_list[end_1-j-1].replace("!", "")
            #print(split_list[end_1-j-1])
            if(split_list[end_1-j-1][0]=='p' or split_list[end_1-j-1][0]=='P'):
                temp_list_bidding[11-j]=0
                continue
            if(split_list[end_1-j-1][0]=='d' or split_list[end_1-j-1][0]=='D'):
                temp_list_bidding[11-j]=36
                continue
            if(split_list[end_1-j-1][0]=='r' or split_list[end_1-j-1][0]=='R'):
                temp_list_bidding[11-j]=37
                continue


            temp = 0
            if(len(split_list[end_1-j-1]) != 2):
                temp = -1
                break
            if(split_list[end_1-j-1][0].isdigit()):
                temp += (int(split_list[end_1-j-1][0])-1)*5
            if(split_list[end_1-j-1][1] == 'C' or split_list[end_1-j-1][1] == 'c'):
                temp += 1
            elif(split_list[end_1-j-1][1] == 'D' or split_list[end_1-j-1][1] == 'd' ):
                temp += 2
            elif(split_list[end_1-j-1][1] == 'H' or split_list[end_1-j-1][1] == 'h'):
                temp += 3
            elif(split_list[end_1-j-1][1] == 'S' or split_list[end_1-j-1][1] == 's'):
                temp += 4
            elif(split_list[end_1-j-1][1] == 'N' or split_list[end_1-j-1][1] == 'n'):
                temp += 5
            else:
                temp = -1
                break
            temp_list_bidding[11-j] = temp
            
        example = temp_list.copy()
        example_without_fea =  without_feature.copy()
        for j in range(12):
            example.append(temp_list_bidding[j])
            example_without_fea.append(temp_list_bidding[j])
        train_data.append(example)
        train_data_without_feature.append(example_without_fea)
        
train_data = np.asarray(train_data)
train_data_without_feature = np.asarray(train_data_without_feature)

In [176]:
train_data = train_data.astype('float')
train_data_without_feature = train_data_without_feature.astype('float')
#np.savetxt('./data/train_data_savefile_back12_orig_30000.txt', train_data, fmt='%f')

In [177]:
train_orig = train_data.copy()
train_without_feature_orig = train_data_without_feature.copy()

In [58]:
train_orig_2 = train_orig.copy()

In [48]:
train_data = train_orig.copy()

In [46]:
train_orig = np.loadtxt('./data/train_data_savefile_back12_orig_30000.txt', dtype=float)

In [99]:
print(train_orig[0])
print(train_orig[1])
#print(train_orig_2[0])
#print(train_orig_2[1])
print(train_data[0])
print(train_data[1])
print(train_data.shape)

[ 1.   0.   0.   0.   1.   3.   3.   4.   3.   0.   0.   0.   0.   0.
  1.   1.   0.   0.   0.   0.   1.   0.   0.   1.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   1.   1.   0.   0.   0.   0.   1.   0.   1.
  1.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.   1.
  0.   0.   0.   0.   1.   1.   7.   2.   2.  12.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   2.5  1.   0.   3.5  1.
  0.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1. ]
[ 1.   0.   0.   0.   1.   3.   3.   4.   3.   0.   0.   0.   0.   0.
  1.   1.   0.   0.   0.   0.   1.   0.   0.   1.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   1.   1.   0.   0.   0.   0.   1.   0.   1.
  1.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.   1.
  0.   0.   0.   0.   1.   1.   7.   2.   2.  12.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   2.5  1.   0.   3.5  1.
  0.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.   9.   0.   0. ]
[ 1.   0.   0.   0.   1.  

In [79]:
print(train_data.shape)
print(train_data[3])
print(train_data[3][2])
print(train_data[3][5:9])
print(train_data[3][65])
print(train_data[3][78:82])
print(train_data[3][85:97])

(341553, 97)
[ 1.   0.   0.   0.   1.   3.   3.   4.   3.   0.   0.   0.   0.   0.
  1.   1.   0.   0.   0.   0.   1.   0.   0.   1.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   1.   1.   0.   0.   0.   0.   1.   0.   1.
  1.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.   1.
  0.   0.   0.   0.   1.   1.   7.   2.   2.  12.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   2.5  1.   0.   3.5  1.
  0.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   1.   4.  36.  37. ]
0.0
[3. 3. 4. 3.]
12.0
[0.  2.5 1.  0. ]
[-1. -1. -1. -1. -1. -1. -1. -1.  1.  4. 36. 37.]


In [97]:
l = len(train_data)
train_data_onlyfeature  = np.zeros((l,22))#only contain 2 3 6 8 12 features in our paper
for i in range(l):
    train_data_onlyfeature[i][0] = train_data[i][2]
    train_data_onlyfeature[i][1:5] = train_data[i][5:9]
    train_data_onlyfeature[i][5] = train_data[i][65]
    train_data_onlyfeature[i][6:10] = train_data[i][78:82]
    train_data_onlyfeature[i][10:23] = train_data[i][85:97]
    


In [100]:
print(train_data_onlyfeature.shape)
print(train_data_onlyfeature[4])

(341553, 22)
[ 0.   3.   3.   4.   3.  12.   0.   2.5  1.   0.  -1.  -1.  -1.  -1.
  1.   4.  36.  37.   0.   0.   6.   9. ]


In [116]:

#train_data = normalize(train_data, axis=0, norm='l2')
normalizer = Normalizer(norm='l2').fit(train_data_onlyfeature)
train_norm = normalizer.transform(train_data_onlyfeature)
print(normalizer.transform(train_data_onlyfeature[0].reshape(1, -1)))
print(train_norm[0])
print(normalizer.transform(train_data_onlyfeature[115000].reshape(1, -1)))
print(train_norm[115000])
print(normalizer)
print(train_data_onlyfeature.shape)
print(train_data_onlyfeature[0])
print(train_data_onlyfeature[115000])

[[ 0.          0.20889319  0.20889319  0.27852425  0.20889319  0.83557275
   0.          0.17407766  0.06963106  0.         -0.06963106 -0.06963106
  -0.06963106 -0.06963106 -0.06963106 -0.06963106 -0.06963106 -0.06963106
  -0.06963106 -0.06963106 -0.06963106 -0.06963106]]
[ 0.          0.20889319  0.20889319  0.27852425  0.20889319  0.83557275
  0.          0.17407766  0.06963106  0.         -0.06963106 -0.06963106
 -0.06963106 -0.06963106 -0.06963106 -0.06963106 -0.06963106 -0.06963106
 -0.06963106 -0.06963106 -0.06963106 -0.06963106]
[[ 0.04941662  0.24708311  0.19766648  0.09883324  0.09883324  0.54358283
   0.12354155  0.04941662  0.02470831  0.         -0.04941662 -0.04941662
  -0.04941662 -0.04941662 -0.04941662 -0.04941662 -0.04941662  0.
   0.19766648  0.39533297  0.59299945  0.        ]]
[ 0.04941662  0.24708311  0.19766648  0.09883324  0.09883324  0.54358283
  0.12354155  0.04941662  0.02470831  0.         -0.04941662 -0.04941662
 -0.04941662 -0.04941662 -0.04941662 -0.04941

In [95]:
'''normalizer = Normalizer(norm='l2').fit(train_data_without_feature)
train_without_feature_norm = normalizer.transform(train_data_without_feature)
print(normalizer.transform(train_data_without_feature[0].reshape(1, -1)))
print(train_without_feature_norm[0])
print(normalizer.transform(train_data_without_feature[115000].reshape(1, -1)))
print(train_without_feature_norm[115000])
print(normalizer)
print(train_data_without_feature.shape)
print(train_data_without_feature[0])
print(train_data_without_feature[115000])'''

"normalizer = Normalizer(norm='l2').fit(train_data_without_feature)\ntrain_without_feature_norm = normalizer.transform(train_data_without_feature)\nprint(normalizer.transform(train_data_without_feature[0].reshape(1, -1)))\nprint(train_without_feature_norm[0])\nprint(normalizer.transform(train_data_without_feature[115000].reshape(1, -1)))\nprint(train_without_feature_norm[115000])\nprint(normalizer)\nprint(train_data_without_feature.shape)\nprint(train_data_without_feature[0])\nprint(train_data_without_feature[115000])"

## Make the training data

In [ ]:

train_data_org = train_data
train_data_backup = train_data

In [139]:
#np.savetxt('./data/train_data_savefile_back12_onlyfeature_30000.txt', train_norm, fmt='%f')
train_data = []
train_data = np.loadtxt('./data/train_data_savefile_back12_onlyfeature_30000.txt', dtype=float)#after normalize
#train_data = np.loadtxt('./data/train_data_savefile_back12_30000.txt', dtype=float)
print(train_data[0])
print(train_data[999])
print(train_data)
print(train_data.shape)

[ 0.        0.208893  0.208893  0.278524  0.208893  0.835573  0.
  0.174078  0.069631  0.       -0.069631 -0.069631 -0.069631 -0.069631
 -0.069631 -0.069631 -0.069631 -0.069631 -0.069631 -0.069631 -0.069631
 -0.069631]
[ 0.023754  0.071262  0.047508  0.071262  0.11877   0.380065  0.035631
  0.        0.        0.11877  -0.023754 -0.023754 -0.023754 -0.023754
 -0.023754 -0.023754 -0.023754  0.11877   0.855145  0.166278  0.
  0.190032]
[[ 0.        0.208893  0.208893 ... -0.069631 -0.069631 -0.069631]
 [ 0.        0.178253  0.178253 ...  0.534758  0.        0.      ]
 [ 0.        0.208893  0.208893 ... -0.069631 -0.069631 -0.069631]
 ...
 [ 0.041739  0.166957  0.041739 ...  0.125218  0.584349  0.709566]
 [ 0.02741   0.10964   0.02741  ...  0.520792  0.        0.548202]
 [ 0.019292  0.077166  0.019292 ...  0.462996  0.        0.540162]]
(341553, 22)


In [143]:
train_data_with = np.loadtxt('./data/train_data_savefile_back12_30000.txt', dtype=float)
train_data_with = np.loadtxt('./data/train_data_savefile_back12_orig_30000.txt', dtype=float)
train_norm_with = normalizer.transform(train_data_with)
print(train_data_with.shape)
print(train_data_with[0])
print(train_norm_with[0])

(341553, 97)
[ 1.   0.   0.   0.   1.   3.   3.   4.   3.   0.   0.   0.   0.   0.
  1.   1.   0.   0.   0.   0.   1.   0.   0.   1.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   1.   1.   0.   0.   0.   0.   1.   0.   1.
  1.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.   1.
  0.   0.   0.   0.   1.   1.   7.   2.   2.  12.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   2.5  1.   0.   3.5  1.
  0.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1. ]
[ 0.05847053  0.          0.          0.          0.05847053  0.1754116
  0.1754116   0.23388214  0.1754116   0.          0.          0.
  0.          0.          0.05847053  0.05847053  0.          0.
  0.          0.          0.05847053  0.          0.          0.05847053
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.05847053  0.05847053  0.
  0.          0.          0.          0.05847053  0.          0.05847053
  0.05847053  0.      

In [ ]:

'''
train_data[:,0] = train_data[:,0]/50
train_data[:,1] = train_data[:,1]/3
train_data[:,4:15] = train_data[:,4:15]/37
train_data[:,16] = train_data[:,16]/4
train_data[:,17:20] = train_data[:,17:20]/13
train_data[:,73:76] = train_data[:,73:76]/13
train_data[:,81] = train_data[:,81]/30
train_data[:,91:94] = train_data[:,91:94]/10
train_data[:,95] = train_data[:,95]/10
train_data[:,96] = train_data[:,96]/10

train_data[:,0] = train_data[:,0]*50
train_data[:,1] = train_data[:,1]*3
train_data[:,4] = train_data[:,4]*2000
train_data[:,5] = train_data[:,5]*4
train_data[:,6:13] = train_data[:,6:13]*13
train_data[:,66:69] = train_data[:,66:69]*13
train_data[:,70] = train_data[:,70]*30
train_data[:,83:86] = train_data[:,83:86]*10
train_data[:,87] = train_data[:,87]*10
'''
print(train_data[0,:])
print(train_data)
print(train_data.shape)
print(type(train_data))

## Make one-hot format labels (But bridge seems to use 13-hot-encoding)

In [131]:
from keras.utils import np_utils
#train_labels[i][1]:
#combats_df.iloc[:,2]

combats_train_hot_label = [[0 for x in range(38)] for y in range(len(train_data))]
print(type(combats_train_hot_label))
t = 0

for i in range(len(bridge_df.iloc[:,:])):
    split_list = bridge_df.iloc[i,4].split('|')
    order = bridge_df.iloc[i,5]
    
    if(len(split_list)%4-order >= 0):
        remain = 1
    else:
        remain = 0
    add_bidding = int(len(split_list)/4) + remain #can cut into how many examples for one bidding seqence
    #print('len:', len(split_list), 'order', order)
    #if(order+1>len(split_list)):
        #combats_train_hot_label[i][35] = 1
        #continue
        
    for ex in range(add_bidding):
        end_1 = ex*4 + order - 1
        #print(end_1)
        split_list[end_1] = split_list[end_1].replace("!", "")
        if(split_list[end_1]=='p' or split_list[end_1]=='p!' or split_list[end_1]=='P' or split_list[end_1]=='P!'):
            combats_train_hot_label[t][0] = 1

        elif(split_list[end_1]=='d' or split_list[end_1]=='d!' or split_list[end_1]=='D' or split_list[end_1]=='D!'):
            combats_train_hot_label[t][36] = 1
        elif(split_list[end_1]=='r' or split_list[end_1]=='r!' or split_list[end_1]=='R' or split_list[end_1]=='R!'):
            combats_train_hot_label[t][37] = 1
        temp_num = 0    
        if (len(split_list[end_1]) == 2 and split_list[end_1][0].isdigit()):
            temp_num = (int(split_list[end_1][0])-1)*5
            if(split_list[end_1][1] == 'C' or split_list[end_1][1] == 'c'):
                temp_num += 1
            elif(split_list[end_1][1] == 'D' or split_list[end_1][1] == 'd' ):
                temp_num += 2
            elif(split_list[end_1][1] == 'H' or split_list[end_1][1] == 'h'):
                temp_num += 3
            elif(split_list[end_1][1] == 'S' or split_list[end_1][1] == 's'):
                temp_num += 4
            elif(split_list[end_1][1] == 'N' or split_list[end_1][1] == 'n'): 
                temp_num += 5
            else:
                temp_num = 0
            combats_train_hot_label[t][temp_num] = 1 
        t += 1
    

#combats_train_hot_label = np_utils.to_categorical(combats_train_label,2)

combats_train_hot_label = np.asarray(combats_train_hot_label)
print(t)





<class 'list'>
341553


In [132]:
print(combats_train_hot_label[114998][:])
print(combats_train_hot_label[114999][:])
print(combats_train_hot_label[115000][:])
print(combats_train_hot_label)
print(combats_train_hot_label.shape)
print(type(combats_train_hot_label))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
(341553, 38)
<class 'numpy.ndarray'>


In [103]:
#np.savetxt('./data/combats_train_hot_label_save_file_back12_30000.txt', combats_train_hot_label, fmt='%d')
combats_train_hot_label = np.loadtxt('./data/combats_train_hot_label_save_file_back12_30000.txt', dtype=int)
print(combats_train_hot_label[0])
print(combats_train_hot_label[999])
print(combats_train_hot_label)
print(combats_train_hot_label.shape)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
(341553, 38)


In [104]:
#combats_train_hot_label = to_categorical(combats_train_hot_label,2)
#combats_train_hot_label = combats_train_hot_label.astype(float)
print(combats_train_hot_label.shape)
print(type(combats_train_hot_label))
print(combats_train_hot_label[0,:])
print(combats_train_hot_label)

(341553, 38)
<class 'numpy.ndarray'>
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


# Build Model

##  Create network

In [105]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras import optimizers
from keras.models import load_model

from keras.datasets import mnist
from keras.utils import np_utils
import keras.callbacks
import os
import os.path
import glob
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras.layers import Embedding
from keras import optimizers

In [218]:
length = len(train_norm[0])
def build_model():
    model = Sequential()
    ###   code here  #####
    #keras.layers.normalization.BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)
    
    model.add(Dense(64,input_shape = (length,))) #need to check the input size, first set the input node to be 128 as matlab do
    model.add(Activation('relu')) #might be leakyRelu, since they used the parameter alpha, still need to check
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    """
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    """ 
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(38,activation='softmax')) #need to check the output size, seems to be 36
    model.summary()
    return model

## Training

In [219]:
############## Start Training ############
# use validation_data=(valid_data,valid_labels) in model.fit !!!!!! #
'''
training parameters
update_dnntype = 2; 
badupdate_dnn = 2; 
explore_first = 1; 
alphaupdate_dnn = 0.1;
batchsizeupdate_dnn = 50;
batchsize = 50;
decayRate = 0.98;
momentum = 0.82;
alpha = 0.83;
startbackprop = 0;
input = 52+36+5;
lsize = 128;
layer = 4;
output = 36;
eta = 0.05;
'''
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
model = build_model()
#sgd = optimizers.SGD(lr=0.2, clipnor0m=1.0,decay = 1e-08) #original 1e-08)
#adadelta = optimizers.Adadelta(lr=1.0, epsilon=None, decay=0.0)
model.compile(optimizer="adadelta"#"adadelta"
            #,loss='categorical_crossentropy'
            ,loss='mae'
            ,metrics=['accuracy'])

record = model.fit(train_norm,combats_train_hot_label
                   ,batch_size=4096 #original 64
                   ,epochs=1000
                   ,validation_split = 0.1
                   ,callbacks=[earlyStopping]
                   ,shuffle = True)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_118 (Dense)            (None, 64)                1472      
_________________________________________________________________
activation_91 (Activation)   (None, 64)                0         
_________________________________________________________________
batch_normalization_91 (Batc (None, 64)                256       
_________________________________________________________________
dropout_91 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_119 (Dense)            (None, 128)               8320      
_________________________________________________________________
activation_92 (Activation)   (None, 128)               0         
_________________________________________________________________
batch_normalization_92 (Batc (None, 128)               512       
__________

In [220]:
###### use "model.predict(testing_data)" to get the results ######
from keras.models import load_model

#model = load_model('bridge_model.h5')
#pred = np.argmax(model.predict(test_data), axis=1).reshape(len(test_data),1)
#pred_array = model.predict(test_data)
#predicted_output_df = pd.DataFrame(pred, columns = ["card output"])  #predicted output
#output = pd.read_csv('bridge_test_output1.csv') #real ouput
score, acc = model.evaluate(test_data_onlyfeature_norm, combats_test_hot_label, batch_size=1024)
print('Test score:', score)
print('Test acc:', acc)
#print(pred)


"""
for i in range(28338):    check = True
    while(check):
        check = False
        for j in range(int(avoid_data[i][16])-1):
            if(pred[i]>=1 and pred[i]<=35):
                if(avoid_data[i][j]>pred[i] and avoid_data[i][j]>=1 and avoid_data[i][j]<=35):
                    pred_array[i][pred[i]] = 0
                    pred[i] = np.argmax(pred_array[i])
                    check = True
            else:
                if(avoid_data[i][j]>pred[i] and (avoid_data[i][j]==36 or avoid_data[i][j]==37)):
                    pred_array[i][pred[i]] = 0
                    pred[i] = np.argmax(pred_array[i])
                    check = True


count = 0
for i in range(28338):
    if(pred[i][0]==np.argmax(combats_test_hot_label[i])):
        count+=1

print(count/28338)#the test accuracy
print(count)


for i in range(test_data):
    pred[i] = np.argmax(model.predict(test_data[i])[前一家叫牌:], axis=1)
"""

56666/56666 [==============================] - 0s 5us/step
Test score: 0.01962980307781698
Test acc: 0.6273956162549267


'\nfor i in range(28338):    check = True\n    while(check):\n        check = False\n        for j in range(int(avoid_data[i][16])-1):\n            if(pred[i]>=1 and pred[i]<=35):\n                if(avoid_data[i][j]>pred[i] and avoid_data[i][j]>=1 and avoid_data[i][j]<=35):\n                    pred_array[i][pred[i]] = 0\n                    pred[i] = np.argmax(pred_array[i])\n                    check = True\n            else:\n                if(avoid_data[i][j]>pred[i] and (avoid_data[i][j]==36 or avoid_data[i][j]==37)):\n                    pred_array[i][pred[i]] = 0\n                    pred[i] = np.argmax(pred_array[i])\n                    check = True\n\n\ncount = 0\nfor i in range(28338):\n    if(pred[i][0]==np.argmax(combats_test_hot_label[i])):\n        count+=1\n\nprint(count/28338)#the test accuracy\nprint(count)\n\n\nfor i in range(test_data):\n    pred[i] = np.argmax(model.predict(test_data[i])[前一家叫牌:], axis=1)\n'

In [103]:
print(pred[1000:1200])

[[36]
 [36]
 [23]
 [ 0]
 [ 4]
 [13]
 [28]
 [28]
 [ 0]
 [ 4]
 [ 0]
 [28]
 [28]
 [ 7]
 [ 7]
 [30]
 [27]
 [27]
 [28]
 [ 6]
 [18]
 [36]
 [ 6]
 [23]
 [36]
 [27]
 [ 2]
 [12]
 [36]
 [27]
 [ 4]
 [ 4]
 [ 7]
 [20]
 [ 4]
 [ 4]
 [14]
 [ 3]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [23]
 [36]
 [36]
 [34]
 [ 4]
 [ 7]
 [34]
 [ 4]
 [ 7]
 [12]
 [ 1]
 [ 8]
 [13]
 [ 1]
 [20]
 [36]
 [23]
 [36]
 [23]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 5]
 [ 0]
 [ 5]
 [ 0]
 [ 0]
 [37]
 [ 0]
 [ 0]
 [37]
 [ 0]
 [37]
 [ 0]
 [29]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 4]
 [37]
 [ 0]
 [ 4]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 5]
 [ 0]
 [34]
 [ 5]
 [36]
 [34]
 [ 7]
 [36]
 [36]
 [30]
 [20]
 [36]
 [10]
 [ 6]
 [10]
 [20]
 [36]
 [18]
 [ 0]
 [ 0]
 [36]
 [23]
 [ 0]
 [ 0]
 [ 4]
 [13]
 [21]
 [ 0]
 [ 4]
 [13]
 [36]
 [27]
 [ 7]
 [ 6]
 [23]
 [ 6]
 [18]
 [ 2]
 [12]
 [17]
 [27]
 [36]
 [27]
 [27]
 [27]
 [ 4]
 [ 9]
 [14]
 [ 4]
 [19]
 [ 3]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [23]
 [ 0]
 [17]
 [36]
 [34]
 [34]
 [ 1]
 [ 8]
 [20]
 [ 1]
 [30

In [ ]:
model.save('./data/bridge_model_back12_800000.h5')

## Plot the training results

In [243]:
score = model.evaluate(train_norm, combats_train_hot_label, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
plt.plot(record.history['acc'],label='acc')
plt.plot(record.history['val_acc'],label='val_acc')
plt.legend(loc='lower right')
plt.show()
plt.plot(record.history['loss'],label='loss')
plt.plot(record.history['val_loss'],label='val_loss')
plt.legend(loc='lower right')
plt.show()

ValueError: Error when checking input: expected dense_251_input to have shape (69,) but got array with shape (97,)

In [ ]:
'''import csv
bridge_df_test = []

bridge_df_test = pd.concat([bridge_dfN_test, bridge_dfS_test], ignore_index=True)
bridge_df_test = pd.concat([bridge_df_test, bridge_dfE_test], ignore_index=True)
bridge_df_test = pd.concat([bridge_df_test, bridge_dfW_test], ignore_index=True)

bridge_df_test.to_csv("bridge_test", encoding='utf-8', index=False)'''


# Testing

## Read testing data

In [182]:
test_df = pd.read_csv('bridge_test.csv') 
#test_df = test_df[:20000]


print(len(test_df.iloc[:,:]))
print(test_df.iloc[:,:].shape)
#print(test_df)

test_data=[]
test_data_without_feature = []

for i in range(len(test_df.iloc[:,:])):
    if test_df.iloc[i,1] == 'none':
        test_df.iloc[i,1] = 0
    elif test_df.iloc[i,1] == 'NS':
        test_df.iloc[i,1] = 1
    elif test_df.iloc[i,1] == 'EW':
        test_df.iloc[i,1] = 2
    elif test_df.iloc[i,1] == 'both':
        test_df.iloc[i,1] = 3
        
    temp_list = []
    without_feature = []
    temp_list.append(bridge_df.iloc[i,0])
    without_feature.append(bridge_df.iloc[i,0])
    temp_list.append(test_df.iloc[i,1])
    without_feature.append(test_df.iloc[i,1])
    temp_list.append(bridge_df.iloc[i,2])
    without_feature.append(bridge_df.iloc[i,2])
    temp_list.append(bridge_df.iloc[i,3])
    without_feature.append(bridge_df.iloc[i,3])
    temp_list.append(bridge_df.iloc[i,5])
    without_feature.append(bridge_df.iloc[i,5])

    split_list6 = test_df.iloc[i,6].split('|')
    for j in range(4):
        temp_list.append(split_list6[j])
    '''
    split_list7 = test_df.iloc[i,7].split('|')
    for j in range(4):
        temp_list.append(split_list7[j])
    '''
    split_list9 = test_df.iloc[i,9].split('|')    
    for j in range(4):
        suit = [0]*13
        for k in range(len(split_list9[j])):
            if split_list9[j][k] == 'A':
                suit[1] = 1
            elif split_list9[j][k] == 'K':
                suit[0] = 1
            elif split_list9[j][k] == 'Q':
                suit[12] = 1
            elif split_list9[j][k] == 'J':
                suit[11] = 1
            elif split_list9[j][k] == 'T':
                suit[10] = 1
            elif split_list9[j][k].isdigit():
                suit[int(split_list9[j][k])] = 1
            else:
                suit[k] = -1
        without_feature = without_feature + suit
        temp_list = temp_list + suit
        
    split_list10 = test_df.iloc[i,10].split('|')
    for j in range(4):
        temp_list.append(split_list10[j])
    temp_list.append(test_df.iloc[i,11])
    
    split_list12 = test_df.iloc[i,12].split('|')
    for j in range(4):
        temp_list.append(split_list12[j])
    split_list13 = test_df.iloc[i,13].split('|')
    for j in range(4):
        temp_list.append(split_list13[j])
    split_list14 = test_df.iloc[i,14].split('|')
    for j in range(4):
        temp_list.append(split_list14[j])
    split_list15 = test_df.iloc[i,15].split('|')
    for j in range(4):
        temp_list.append(split_list15[j])
    
    temp_list.append(test_df.iloc[i,16])
    temp_list.append(test_df.iloc[i,17])
    if(test_df.iloc[i,18] == 'N'):
        temp_list.append(0)
    elif(test_df.iloc[i,18] == 'S'):
        temp_list.append(1)
    elif(test_df.iloc[i,18] == 'E'):
        temp_list.append(2)
    elif(test_df.iloc[i,18] == 'W'):
        temp_list.append(3)
    else:
        temp_list.append(-1)        
              
    order = test_df.iloc[i,5]    
    split_list = test_df.iloc[i,4].split('|')
    if(len(split_list)%4-order >= 0):
        remain = 1
    else:
        remain = 0
    add_bidding = int(len(split_list)/4) + remain #can cut into how many examples for one bidding seqence    

    for ex in range(add_bidding):
        temp = 0
        temp_list_bidding =[-1]*12
        end_1 = ex*4 + order - 1 # split_list[end_1] is the bidding choice of this player in this round 
        #print('end_1: ', end_1)
        #lenth_1 = len(split_list)-1
        for j in range(end_1):
            if(j>11):
                break
            split_list[end_1-j-1] = split_list[end_1-j-1].replace("!", "")
            #print(split_list[end_1-j-1])
            if(split_list[end_1-j-1][0]=='p' or split_list[end_1-j-1][0]=='P'):
                temp_list_bidding[11-j]=0
                continue
            if(split_list[end_1-j-1][0]=='d' or split_list[end_1-j-1][0]=='D'):
                temp_list_bidding[11-j]=36
                continue
            if(split_list[end_1-j-1][0]=='r' or split_list[end_1-j-1][0]=='R'):
                temp_list_bidding[11-j]=37
                continue


            temp = 0
            if(len(split_list[end_1-j-1]) != 2):
                temp = -1
                break
            if(split_list[end_1-j-1][0].isdigit()):
                temp += (int(split_list[end_1-j-1][0])-1)*5
            if(split_list[end_1-j-1][1] == 'C' or split_list[end_1-j-1][1] == 'c'):
                temp += 1
            elif(split_list[end_1-j-1][1] == 'D' or split_list[end_1-j-1][1] == 'd' ):
                temp += 2
            elif(split_list[end_1-j-1][1] == 'H' or split_list[end_1-j-1][1] == 'h'):
                temp += 3
            elif(split_list[end_1-j-1][1] == 'S' or split_list[end_1-j-1][1] == 's'):
                temp += 4
            elif(split_list[end_1-j-1][1] == 'N' or split_list[end_1-j-1][1] == 'n'):
                temp += 5
            else:
                temp = -1
                break
            temp_list_bidding[11-j] = temp
            
        example = temp_list.copy()
        example_without_fea =  without_feature.copy()
        for j in range(12):
            example.append(temp_list_bidding[j])
            example_without_fea.append(temp_list_bidding[j])
        test_data.append(example)
        test_data_without_feature.append(example_without_fea)

test_data = np.asarray(test_data)
test_data_without_feature = np.asarray(test_data_without_feature)
test_data = test_data.astype('float')
test_data_without_feature = test_data_without_feature.astype('float')

20000
(20000, 19)


In [183]:
print(test_data.shape)
print(test_data[5])
print(bridge_df.iloc[5,1])
l = len(test_data)
test_data_onlyfeature  = np.zeros((l,22))#only contain 2 3 6 8 12 features in our paper
for i in range(l):
    test_data_onlyfeature[i][0] = test_data[i][2]
    test_data_onlyfeature[i][1:5] = test_data[i][5:9]
    test_data_onlyfeature[i][5] = test_data[i][65]
    test_data_onlyfeature[i][6:10] = test_data[i][78:82]
    test_data_onlyfeature[i][10:23] = test_data[i][85:97]


test_data_onlyfeature = test_data_onlyfeature.astype('float')
print(test_data_onlyfeature.shape)
print(test_data_onlyfeature[1])

(56666, 97)
[ 1.  0.  0.  0.  1.  4.  5.  3.  1.  0.  0.  0.  1.  0.  0.  1.  0.  0.
  1.  0.  0.  1.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  2.  4.  0.  4. 10.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  1.  0.  0.  1.  3.  0.  1.  5.  2.  0. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.  0.  4.]
EW
(56666, 22)
[ 0.  4.  4.  3.  2.  2.  1.  1.  0.  0. -1. -1. -1. -1. -1.  4.  0.  7.
  0.  8.  0. 10.]


In [188]:
print(test_data_without_feature[28335])
print(test_data_without_feature[28336])
print(test_data_without_feature[28337])
print(test_data_without_feature)
print(test_data_without_feature.shape)

[ 9.  2.  0.  1.  1.  0.  1.  0.  0.  0.  1.  1.  0.  0.  1.  0.  1.  0.
  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.
  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[ 9.  2.  0.  1.  1.  0.  1.  0.  0.  0.  1.  1.  0.  0.  1.  0.  1.  0.
  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.
  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1.  4.  0. 10.  0.]
[ 9.  2.  0.  1.  1.  0.  1.  0.  0.  0.  1.  1.  0.  0.  1.  0.  1.  0.
  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.
  0.  0.  0. -1. -1. -1. -1.  4.  0. 10.  0. 11.  0. 19.  0.]
[[ 1.  0.  0. ...  4.  0.  7.]
 [ 1.  0.  0. ...  8.  0. 10.]
 [ 1.  0.  0. ... 15.  0. 17.]
 ...
 [24.  0.  0. ... -1. -1.  4.]
 [24.  0.  0. ... 10.  0. 11

In [187]:
#ttt = np.append(traintest_data)
#avoid_data = test_data
test_norm = normalizer.transform(test_data)
test_without_feature_norm = normalizer.transform(test_data_without_feature)
print(test_data, test_norm,test_data.shape)
print(test_data_without_feature, test_without_feature_norm)

[[ 1.  3.  0. ...  4.  0.  7.]
 [ 1.  3.  0. ...  8.  0. 10.]
 [ 1.  3.  0. ... 15.  0. 17.]
 ...
 [24.  0.  0. ... -1. -1.  4.]
 [24.  0.  0. ... 10.  0. 11.]
 [24.  0.  0. ... 19.  0.  0.]] [[ 0.07980869  0.23942607  0.         ...  0.31923475  0.
   0.55866082]
 [ 0.0561656   0.16849679  0.         ...  0.44932477  0.
   0.56165596]
 [ 0.03477341  0.10432022  0.         ...  0.52160111  0.
   0.59114792]
 ...
 [ 0.85847842  0.          0.         ... -0.03576993 -0.03576993
   0.14307974]
 [ 0.75949272  0.          0.         ...  0.3164553   0.
   0.34810083]
 [ 0.6518557   0.          0.         ...  0.51605243  0.
   0.        ]] (56666, 97)
[[ 1.  3.  0. ...  4.  0.  7.]
 [ 1.  3.  0. ...  8.  0. 10.]
 [ 1.  3.  0. ... 15.  0. 17.]
 ...
 [24.  0.  0. ... -1. -1.  4.]
 [24.  0.  0. ... 10.  0. 11.]
 [24.  0.  0. ... 19.  0.  0.]] [[ 0.10101525  0.30304576  0.         ...  0.40406102  0.
   0.70710678]
 [ 0.06225728  0.18677184  0.         ...  0.49805825  0.
   0.62257281]
 [ 0.0

In [171]:
test_data_onlyfeature_norm = normalizer.transform(test_data_onlyfeature)
print(test_data_onlyfeature_norm)

[[ 0.          0.35777088  0.35777088 ...  0.35777088  0.
   0.62609903]
 [ 0.          0.23693955  0.23693955 ...  0.4738791   0.
   0.59234888]
 [ 0.          0.14186538  0.14186538 ...  0.53199518  0.
   0.60292787]
 ...
 [ 0.          0.          0.41442434 ... -0.08288487 -0.08288487
   0.33153947]
 [ 0.          0.          0.26259023 ...  0.52518046  0.
   0.57769851]
 [ 0.          0.          0.18639564 ...  0.70830342  0.
   0.        ]]


# Make testing data labels

In [1]:
combats_test_hot_label = [[0 for x in range(38)] for y in range(len(test_data))]
print(type(combats_test_hot_label))
t = 0

for i in range(len(test_df.iloc[:,:])):
    split_list = test_df.iloc[i,4].split('|')
    order = test_df.iloc[i,5]
    
    if(len(split_list)%4-order >= 0):
        remain = 1
    else:
        remain = 0
    add_bidding = int(len(split_list)/4) + remain #can cut into how many examples for one bidding seqence
    #print('len:', len(split_list), 'order', order)
    #if(order+1>len(split_list)):
        #combats_train_hot_label[i][35] = 1
        #continue
        
    for ex in range(add_bidding):
        end_1 = ex*4 + order - 1
        #print(end_1)
        split_list[end_1] = split_list[end_1].replace("!", "")
        if(split_list[end_1]=='p' or split_list[end_1]=='p!' or split_list[end_1]=='P' or split_list[end_1]=='P!'):
            combats_test_hot_label[t][0] = 1

        elif(split_list[end_1]=='d' or split_list[end_1]=='d!' or split_list[end_1]=='D' or split_list[end_1]=='D!'):
            combats_test_hot_label[t][36] = 1
        elif(split_list[end_1]=='r' or split_list[end_1]=='r!' or split_list[end_1]=='R' or split_list[end_1]=='R!'):
            combats_test_hot_label[t][37] = 1
        temp_num = 0    
        if (len(split_list[end_1]) == 2 and split_list[end_1][0].isdigit()):
            temp_num = (int(split_list[end_1][0])-1)*5
            if(split_list[end_1][1] == 'C' or split_list[end_1][1] == 'c'):
                temp_num += 1
            elif(split_list[end_1][1] == 'D' or split_list[end_1][1] == 'd' ):
                temp_num += 2
            elif(split_list[end_1][1] == 'H' or split_list[end_1][1] == 'h'):
                temp_num += 3
            elif(split_list[end_1][1] == 'S' or split_list[end_1][1] == 's'):
                temp_num += 4
            elif(split_list[end_1][1] == 'N' or split_list[end_1][1] == 'n'): 
                temp_num += 5
            else:
                temp_num = 0
            combats_test_hot_label[t][temp_num] = 1 
        t += 1
    

#combats_train_hot_label = np_utils.to_categorical(combats_train_label,2)

combats_test_hot_label = np.asarray(combats_test_hot_label)
#combats_test_hot_label = combats_test_hot_label.astype(float)
print(t)





NameError: name 'test_data' is not defined

In [78]:
print(combats_test_hot_label[28335])
print(combats_test_hot_label[28336])
print(combats_test_hot_label[28337])

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


In [ ]:


test_data[:,0] = test_data[:,0]/50
test_data[:,1] = test_data[:,1]/3
test_data[:,4:15] = test_data[:,4:15]/37
test_data[:,16] = test_data[:,16]/4
test_data[:,17:20] = test_data[:,17:20]/13
test_data[:,73:76] = test_data[:,73:76]/13
test_data[:,81] = test_data[:,81]/30
test_data[:,91:94] = test_data[:,91:94]/10
test_data[:,95] = test_data[:,95]/10
test_data[:,96] = test_data[:,96]/10


print(test_data.shape)
print(type(test_data))
print(test_data[0,:])
print(test_data)

## Predict results

In [ ]:
output = np.asarray(output)
test_data = np.asarray(test_data)
predicted_output_df = np.asarray(predicted_output_df)

temp = [[0 for x in range(38)] for y in range(20000)]

for i in range(len(output)):
    temp[i][int(output[i])] = 1



score= model.evaluate(test_data, ground[:19999], verbose=0)

In [ ]:
print(a)

## Print the result to csv file

In [ ]:
df.to_csv('result.csv',index_label = "index")